Another method I would like to try is a neural network. 
Here we can see that we have 127 input features and we know we have 3 output classes so we should use 
65 hidden neurons as in accordance with this stack overflow post 
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

In [1]:

# import os
# os.environ["KERAS_BACKEND"] = "torch"
import numpy as np
import pandas as pd
import sklearn.impute
import sklearn.neighbors 
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import neural_network
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, make_scorer
# import keras 
# from keras import layers
# from keras import ops 


import torch
from torch import nn
import torch.utils.data as tud
from torchvision import datasets
from torchvision.transforms import ToTensor






In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [3]:
### Import and Clean Data
# Import Training Data
training_data= pd.read_csv("./Data/training_data.csv", sep=";", decimal=',')

# features and target split 
data_X = training_data.loc[:, training_data.columns != "Class"]
data_X = data_X.loc[:, data_X.columns != "Perform"]

data_Class = training_data.loc[:, "Class"]
data_Perform = training_data.loc[:,"Perform"]

# Change Class labels 
# Define the expected class mapping
label_mapping = {0: 2, 1: 0, -1 : 1}  # Change according to your expected class order

# Apply the mapping to the target variable
data_Class = data_Class.map(label_mapping)

# Take the names of only the numeric columns, i.e. columns other than group 
numeric_col =data_X.iloc[:,1:].columns # data without groups

# Scale the data 
scaler=preprocessing.StandardScaler()
data_X[numeric_col] = scaler.fit_transform(data_X[numeric_col])
data_X.head()

# Add Dummy Variables for Group
print(data_X.shape)
data_X = pd.get_dummies(data_X,"Group_",columns=["Group"])
print("after group dv")
print(data_X.shape)



(8000, 117)
after group dv
(8000, 127)


In [4]:
# Impute Data 
imp = sklearn.impute.SimpleImputer(strategy="median")
data_Xsimple = imp.fit_transform(data_X)
imp = sklearn.impute.KNNImputer(n_neighbors=3,weights = "uniform")
data_XkNN = imp.fit_transform(data_X)

In [5]:
# Split Given Training Data to Test/Train Split 
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(data_Xsimple, data_Class, test_size=0.25, random_state=30)

In [ ]:
#Custom scorer Template 
def avg_err_cost(y_true, y_pred):
    # Define the desired rows
    row1 = [0, 1, 2]
    row2 = [1, 0, 1]
    row3 = [2, 1, 0]

    cost_matrix = np.array([row1, row2, row3]) 
    confu_matrix = confusion_matrix(y_true, y_pred)
    acc = 1 - np.sum(confu_matrix * cost_matrix)/len(y_true)
    return  acc

custom_scorer = make_scorer(avg_err_cost)



In [9]:
batch_size = 1000   # arbitrarily choosen 

# Create Data loaders 

print(data_Xsimple.shape)
print(data_Class.shape)
targets = torch.tensor(data_Class)
inputs = torch.tensor(data_Xsimple)
# print(t)
# print(t.shape)
# print(t2)
# print(t2.shape)
dataset = tud.TensorDataset(inputs, targets) # load  train data in pytorch form TensorDataset(inputs, targets)
test_ds = tud.TensorDataset(inputs,targets) # load test data in pytorch form 

train_data_loader = tud.DataLoader(dataset, batch_size=batch_size) # allows for batchable content
test_data_loader = tud.DataLoader(test_ds, batch_size=batch_size)




(8000, 127)
(8000,)


The idea for the neural network is to have two hidden layers that will have the average of the target and the previous layers number of nodes 

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(127,65 ), # Input -> H1: go from 127 to 65 nodes  
            torch.nn.ReLU(), # activation
            torch.nn.Linear(65,34 ), #  H1 -> H2: another layer of 65 nodes to 34 
            torch.nn.ReLU(), # activation 
            torch.nn.Linear(34,3), # H2 -> output
            )
    
    def forward(self,x):
        raw_out_values = self.linear_relu_stack(x)
        return raw_out_values
    



In [11]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [20]:
model = NeuralNetwork().to(dtype=torch.float32)
print(model) # print the model structure 

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=127, out_features=65, bias=True)
    (1): ReLU()
    (2): Linear(in_features=65, out_features=34, bias=True)
    (3): ReLU()
    (4): Linear(in_features=34, out_features=3, bias=True)
  )
)


I think in the end I will get class predictions of the values $\{0,1,2\}$ so I think i will need to rename the data accordingly so that i properly predict the outputs 

In [13]:
## Test 
X = torch.rand(127, device=device)
logits = model(X)
print(logits)
pred_prob = nn.Softmax(dim=0)(logits)
print(pred_prob)
y_pred = pred_prob.argmax(0)
print(y_pred)

tensor([-0.0457, -0.1670,  0.0924], device='mps:0', grad_fn=<LinearBackward0>)
tensor([0.3296, 0.2920, 0.3784], device='mps:0', grad_fn=<SoftmaxBackward0>)
tensor(2, device='mps:0')


In [58]:
# Create a custom pytorch loss function based on above 
def custom_loss(y_true, y_pred):
    y_true = y_true.detach().numpy()
    y_pred = y_pred.detach().numpy()
    # print(y_true.shape)
    # print(y_pred.shape)
    row1 = [0, 1, 2]
    row2 = [1, 0, 1]
    row3 = [2, 1, 0]

    cost_matrix = np.array([row1, row2, row3]) 
    confu_matrix = confusion_matrix(y_true, y_pred)
    err = np.sum(confu_matrix * cost_matrix)/len(y_true)
    return err 

loss_fn = nn.CrossEntropyLoss()
# Define optimizer using stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [64]:
# Out of the box training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # print(type(X))
        # print(type(y))
        X, y = X.to(dtype=torch.float32), y.to(dtype=torch.float32) # fixed float64 bug

        # Compute prediction
        raw_out_val = model(X)
        # print("rawoutshape")
        # print(raw_out_val.shape)
        pred_prob = nn.Softmax(dim=1)(raw_out_val)  
        # print("predprob shape")
        # print(pred_prob.shape)
        # pred = pred_prob.argmax(1)# dont need this bc cross entropy loss expects probabilities
        # print("pred shape")
        # print(pred.shape)
        # print("next ")

        # Compute loss
        loss = loss_fn(pred_prob,y.long())

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [66]:
# out of the box testing function 
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(dtype=torch.float32), y.to(dtype=torch.float32) # fixed float64 bug
            # Compute prediction
            raw_out_val = model(X)
            pred = nn.Softmax(dim=1)(raw_out_val)
            #pred = pred_prob.argmax(1) # dont need this bc cross entropy loss expects probabilities 
            test_loss += loss_fn(pred,y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [70]:
epochs = 1000

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, model, loss_fn, optimizer)
    test(train_data_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.090176  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.5%, Avg loss: 1.090265 

Epoch 2
-------------------------------
loss: 1.090101  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 1.090188 

Epoch 3
-------------------------------
loss: 1.090026  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.5%, Avg loss: 1.090110 

Epoch 4
-------------------------------
loss: 1.089951  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.5%, Avg loss: 1.090033 

Epoch 5
-------------------------------
loss: 1.089877  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 1.089956 

Epoch 6
-------------------------------
loss: 1.089802  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 1.089879 

Epoch 7
-------------------------------
loss: 1.089727  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 1.089802 

Epoch 8
-------------------------------
loss: 1.089653  [ 1000/ 8000]
Test Error: 
 Accuracy: 43.7%, Avg loss: 1.089726 

Epoch 9
----------------